In [1]:
#Shishir Khanal
#CMU-Optimal Controls from Jack Manchester
#Wahba's Problem for Pose Estimation
#Given a bunch of vectors to known features in the environment, determine the robot's attitude

In [2]:
import Pkg; Pkg.activate(@__DIR__); Pkg.instantiate()
Pkg.add("LinearAlgebra")
Pkg.add("ForwardDiff")
using LinearAlgebra
using ForwardDiff

  Activating new project at `~/Documents/Optimal_Control/Sims/Pose_Estimation`
  No Changes to `~/Documents/Optimal_Control/Sims/Pose_Estimation/Project.toml`
  No Changes to `~/Documents/Optimal_Control/Sims/Pose_Estimation/Manifest.toml`
    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
    Updating `~/Documents/Optimal_Control/Sims/Pose_Estimation/Project.toml`
  [37e2e46d] + LinearAlgebra
    Updating `~/Documents/Optimal_Control/Sims/Pose_Estimation/Manifest.toml`
  [56f22d72] + Artifacts
  [8f399da3] + Libdl
  [37e2e46d] + LinearAlgebra
  [e66e0078] + CompilerSupportLibraries_jll v0.5.2+0
  [4536629a] + OpenBLAS_jll v0.3.20+0
  [8e850b90] + libblastrampoline_jll v5.1.1+0
   Resolving package versions...
    Updating `~/Documents/Optimal_Control/Sims/Pose_Estimation/Project.toml`
  [f6369f11] + ForwardDiff v0.10.35
    Updating `~/Documents/Optimal_Control/Sims/Pose_Estimation/Manifest.toml`
  [d360d2e6] + ChainRulesCore v1.15.7
  [9e997f

In [3]:
function hat(v)
    return [0 -v[3] v[2];
            v[3] 0 -v[1];
            -v[2] v[1] 0]
end

hat (generic function with 1 method)

In [4]:
function L(q)
    s = q[1]
    v = q[2:4]
    L = [s -v';
         v s*I+hat(v)]
    return L
end

L (generic function with 1 method)

In [5]:
function R(q)
    s = q[1]
    v = q[2:4]
    R = [s -v';
         v s*I-hat(v)]
    return R
end

R (generic function with 1 method)

In [6]:
T = Diagonal([1; -ones(3)])
H = [zeros(1,3); I];

In [7]:
function G(q)
    G = L(q)*H
end

function Q(q)
    return H'*(R(q)*L(q))*H
end

Q (generic function with 1 method)

In [8]:
#Generate random quaternion
qtrue = randn(4)
qtrue = qtrue / norm(qtrue)

Qtrue = Q(qtrue) #Generate Equivalent Rotation matrix

3×3 Matrix{Float64}:
  0.820888   0.159233   -0.0453622
  0.159233   0.858441    0.0403274
 -0.0453622  0.0403274   0.988512

In [9]:
#Generate data
vN = randn(3,10) #Generate some random world-frame vectors

#normalize
for k = 1:10
    vN[:,k] .= vN[:,k]./norm(vN[:,k])
end

vB = Qtrue'*vN #generate body-frame vectors

3×10 Matrix{Float64}:
  0.0687165  -0.5634    -0.563542  …  -0.276964  -0.449785   -0.340816
  0.521291   -0.107645   0.37903       0.629689  -0.856557   -0.399521
 -0.708976   -0.637138   0.27782       0.408044  -0.0114169   0.830892

In [10]:
function residual(q)
    r = vN - Q(q)*vB
    return r[:]
end

residual (generic function with 1 method)

In [11]:
#Random initial guess
q = randn(4)
q = q/norm(q)

4-element Vector{Float64}:
 -0.06874200393221411
 -0.18378280403759029
  0.6797832746840458
 -0.7066775200155295

In [13]:
#Gauss-Newton
ϕ = ones(3)
iter = 0
while maximum(abs.(ϕ)) > 1e-8
    r = residual(q)
    dr = ForwardDiff.jacobian(residual, q)
    ∇r = dr*G(q)
    ϕ = -(∇r'*∇r)\(∇r'*r) #3-parameter update computed with gauss-newton
    q = L(q)*[sqrt(1-ϕ'*ϕ); ϕ] #multiplicative update applied to q
    iter += 1
end

LoadError: DomainError with -65.41017152601252:
sqrt will only return a complex result if called with a complex argument. Try sqrt(Complex(x)).

In [14]:
iter

0

In [15]:
q-qtrue

4-element Vector{Float64}:
  0.8444501476723965
 -0.48304210481619764
  0.9458278743375476
 -0.7824682591625325

In [16]:
q+qtrue

4-element Vector{Float64}:
 -0.9819341555368246
  0.11547649674101707
  0.41373867503054396
 -0.6308867808685266